# Cross Entropy Gradient 计算

reference 
[softmax和Cross Entropy 导数推导](https://www.cnblogs.com/wuliytTaotao/p/10787510.html)

先根据结论

目标概率分布为p， 当前logits softmax的概率分布为q

则cross entropy loss对于logits的导数为：

grad_logits = q - p

## Pytorch求导

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(0)

bs = 1
N = 10 #分类

logits = torch.randn(1, N, requires_grad=True)
labels = torch.randint(high = N, size=(1, bs))[0]
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits, labels)
loss.backward()
logits.grad

tensor([[ 0.3636,  0.0581,  0.0088,  0.1375,  0.0263,  0.0192, -0.8834,  0.1800,
          0.0379,  0.0520]])

## 手动求logits导 

In [2]:
y_hat = torch.zeros(bs, N)
y_hat[0,labels] = 1
print(y_hat)

y = F.softmax(logits, dim=1)[0]

grad_logits = y-y_hat
print(grad_logits)
      

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

tensor([[ 0.3636,  0.0581,  0.0088,  0.1375,  0.0263,  0.0192, -0.8834,  0.1800,
          0.0379,  0.0520]], grad_fn=<SubBackward0>)

# 展开Cross Entropy一步步求导

In [3]:
# bs = 1
# N = 10 #分类
# logits = torch.randn(1, N, requires_grad=True)
# labels = torch.randint(high = N, size=(1, bs))[0]
p = torch.zeros(bs, N)
p[0, labels] = 1

In [4]:
# 前向计算

q = F.softmax(logits , dim = 1)
print(q)

entropy = -p * q.log()
print(entropy)

loss = entropy.mean()
print(loss)

tensor([[0.3636, 0.0581, 0.0088, 0.1375, 0.0263, 0.0192, 0.1166, 0.1800, 0.0379,
         0.0520]], grad_fn=<SoftmaxBackward0>)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1494, 0.0000, 0.0000,
         0.0000]], grad_fn=<MulBackward0>)

tensor(0.2149, grad_fn=<MeanBackward0>)

In [5]:
# 反向计算
# 交叉熵求导

d_entropy = -p/q
print(d_entropy)

tensor([[-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -8.5798, -0.0000,
         -0.0000, -0.0000]], grad_fn=<DivBackward0>)

In [6]:
# 反向计算
# softmax求导
d_e = q 

d_logits = torch.diag(d_e[0]) - d_e.t() @ d_e
# print(d_logits)

In [7]:
# 反向计算
# 最终logits的梯度为
d_logits =  d_entropy @ d_logits
print(d_logits)

# 结果与pytorch一致

tensor([[ 0.3636,  0.0581,  0.0088,  0.1375,  0.0263,  0.0192, -0.8834,  0.1800,
          0.0379,  0.0520]], grad_fn=<MmBackward0>)

## 结论

先说CE的梯度有两种计算方式：

1. CrossEntropy可以一次性的得到logits的梯度
2. 如果从 dCE ->  d softmax -> d logits， 会加大计算量。

所以这是为什么CE的调用里是传logits（方式1）

而不是外部算softmax，再传入到CE里，这样会加大求导的计算量（方式2）

# 补充

在上述实现里我们直接使用了CE和softmax的求导公式。
这里我们来手动求梯度

## CE的梯度推导

```
给定
P=[0,   1,     0] # 目标概率
Q=[0.1, 0.7, 0.2]

开始计算CE

CE = - (p1 logq1 + p2 logq2 + p3 logq3)
那么求导为：
dce/dq1 = -（p1 log q1)'  #q2,q3与q1无关
由于(logq1)' = 1/q1
所以dce/dq1 = -（p1/q1)
```

In [8]:
p = torch.tensor([0.0, 1.0, 0.0])
q = torch.tensor([0.1, 0.7, 0.2])

grad_q = - p/q
print(grad_q)

tensor([-0.0000, -1.4286, -0.0000])

## softmax梯度求导

```
给定 logits
logits = [l1,l2,l3]
[q1, q2, q3] = softmax(logits)

展开为：

q1 = e(l1) / (e(l1)+e(l2)+e(l3)) = e(l1) / sum(l)
q2 = e(l2) / (e(l1)+e(l2)+e(l3)) = e(l2) / sum(l)
q3 = e(l3) / (e(l1)+e(l2)+e(l3)) = e(l2) / sum(l)

除法导数为：
[e(x)] = [f(x)/g(x)]' = (f(x)'g(x) - f(x)g(x)' ) / g(x)^2

那么
d q1 / d l1 = (e(l1)' sum(l) - el1 sum(l)') / sum(l)^2
            = e(l1)sum(l)/sum(l)^2 -  (el1 el1)/sum(l)^2
            = q1 - q1*q1
d q1 / d l2 = (e(l1)' sum(l) - el1 sum(l)') / sum(l)^2
            = 0 -  (el1 el2)/sum(l)^2
            = 0 - q1*q2
```

此时可以写出

```
dq/dl = 

q1-q1*q1  0-q1*q2  0-q1*q3
0 -q2*q1  q2-q2*q2 0-q2*q3
0 -q3*q1  0 -q3*q2 q3-q3*q3

= diag(q) - q.t() * q

```

In [15]:
logits = torch.tensor([1.0, 2.0, 3.0]) # 1x3
q = F.softmax(logits, dim=0)           # 1x3
grad_logits = torch.diag(q) - q.t()@q  # q.t() [3x1], q [1x3] -> 3x3
print(grad_logits)

tensor([[-0.4205, -0.5105, -0.5105],
        [-0.5105, -0.2658, -0.5105],
        [-0.5105, -0.5105,  0.1547]])

# CE+softmax 推导

```
dce/dq =

-p1/q1, -p2/q2  -p3/q3

dq/dl = 

q1-q1*q1  0-q1*q2  0-q1*q3
0 -q2*q1  q2-q2*q2 0-q2*q3
0 -q3*q1  0 -q3*q2 q3-q3*q3



dq/dl1 = 

q1-q1*q1  
0 -q2*q1  
0 -q3*q1  


先算第一个元素
dce/dq dq/dl =

dce/dl1 = 

= (-p1/q1) * （q1-q1*q1）+ (-p2/q2)*（-q2*q1） + (-p3/q3) *（-q3*q1）
= -p1 * (1-q1）+ p2*q1 + p3*q1
= -p1 + [p1q1 + p2q1 + p3q1]
= -p1 + (p1+p2+p3)q1
= -p1 + 1 q1
= q1 - p1
```

In [49]:
## CE+Softmax推导
p = torch.tensor([0.0, 1.0, 0.0])
logits = torch.tensor([1.0, 2.0, 3.0])
q = F.softmax(logits, dim=0)

dce_dq = -(p/q)
dq_dl = torch.diag(q) - torch.outer(q,q)
dce_dl = dce_dq @ dq_dl
print(dce_dl)
print(q-p)

tensor([ 0.0900, -0.7553,  0.6652])

tensor([ 0.0900, -0.7553,  0.6652])